the purpose of this notebook is to take the big dataframe created in 07.01-baseline_data_merging, and make an aggregate we can use for our baseline "worst case" scenario model. This will then be fed into R, where we will use the MICE package to impute data

changelog:

* 4/16/19: added newagg
* 4/17/19: reformatted the ordering of how code runs, and variable names. added aggregation #3.
* 4/19/19: changed the standardizing so that log(x+1) is now applied prior to standardization. also removed ordinal variables from standardizing algorithm and concat them in later with median 0 and iqr 1 so standardize value is either 0 or 1. values from ordinal are not log transformed.


06/14/19:
# prior to this point my pipeline:
1. first median standardized
2. aggregated
3. converted to 2class
4. train/test split
5. imputed


# a big change will happen in this notebook, I will first:
1. convert to two class (c-/abshort &c+/ablong)
2. split the train and test set
3. median standardize
4. aggregate
5. impute
BEFORE I AGGREGATE and median standardize, as some information may be leaking from the train/test set as i had it previously.

## last run: 6/14/19: sensitivity analysis 3day timewindow

In [40]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import numpy as np
import glob

from sklearn.externals.joblib import Memory
from sklearn.model_selection import train_test_split
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 17.2 ms


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The 'cachedir' parameter has been deprecated in version 0.12 and will be removed in version 0.14.
You provided "cachedir='/tmp'", use "location='/tmp'" instead.
  # This is added back by InteractiveShellApp.init_path()


In [41]:
#cohort import

os.chdir('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling') #use to change working directory
wd= os.getcwd() #'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling'


most_updated_patient_df= "04042019"
final_pt_df2 = pd.read_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_final_pt_df2.csv'%(most_updated_patient_df), index_col=0) #only for patients with minimum vitals, 14478 icustay_id
patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 96 ms


In [42]:
len(patients)

11987

time: 1.91 ms


In [43]:

# # ##24 hr sensitivity
# # #importing in all clinical_variable files
# lower_window=0
# upper_window=1
# time_col="charttime"
# time_var="t_0"
# folder="24_hr_window"
# timewindowdays="24"
# date= '09062019'
# patient_df= final_pt_df2

# #48 hr sensitivity
# lower_window=0
# upper_window=2
# time_var="t_0"
# folder="48_hr_window"
# timewindowdays="48"
# date='16052019'
# time_col="charttime"
# time_var= 't_0'
# patient_df= final_pt_df2

#72 hr elixhauser-redo
date='11062019'
lower_window=0
upper_window=3
timewindowdays="72"
folder="{}_hr_window".format(timewindowdays)
time_col="charttime"
time_var= 't_0'
patient_df= final_pt_df2


os.chdir(r'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/processed/')
big_df= pd.read_csv(Path(wd+'/data/processed/merged/{}_longdf_preImp_{}.csv'.format(date,timewindowdays),  index_col=0))
big_df= big_df.reset_index(drop=True).iloc[:,1:]

#quick housekeeping addition to accomidate older generated data
if len(big_df.loc[big_df['label']=="pao2/fio2",'label'])>1:
    big_df.loc[big_df['label']=="pao2/fio2",'label']="pao2fio2ratio"

time: 7.03 s


In [44]:
## last minute data cleaning/formatting

# #removing firstpos else neg ssc col
# big_df=big_df.loc[:,list(big_df.columns!="first_pos_else_neg_ssc")]
# #changing pao2/fio2 ratio label

len(big_df.loc[big_df['label']=="pao2/fio2",'label'])>1

False

time: 178 ms


In [45]:
#overview of all variables and formats
big_df.groupby('label')['value'].describe().sort_values('unique')

,count,unique,top,freq
label,,,,
leukocyte,11016,2,Neg/Not_tested,9554
vasopressin,12766,2,0.0,10735
rrt,11017,2,0.0,10176
phenylephrine,18415,2,0.0,9382
cancer_elix,11017,2,0.0,9660
o2_flow,11155,2,0.0,10550
norepinephrine,23224,2,1.0,14334
nitrite,11016,2,Neg/Not_tested,10725
dobutamine,11753,2,0.0,10858


time: 1.03 s


## initial data prep

In [46]:
#convert to two class

final_pt_df2['final_bin'].unique()
two_classes=['C_neg/A_partial','C_pos/A_full']
two_class_icu=final_pt_df2.loc[final_pt_df2.loc[:,"final_bin"].isin(two_classes),['icustay_id','subject_id','final_bin']]


time: 7.48 ms


In [47]:
big_df=big_df.loc[big_df['icustay_id'].isin(list(two_class_icu['icustay_id'])),:].copy()

time: 729 ms


In [48]:
len(big_df)#7015776 -> ~3000000
big_df['icustay_id'].nunique() #14181 ->7588

5877

time: 25.4 ms


### roundabout way of sampling train/test set so that each subject is only in either train or test:

In [49]:
# label each subject_id with the max of the two classes. 
two_class_pt=two_class_icu.copy()
two_class_pt['final_bin']=pd.factorize(two_class_pt['final_bin'])[0]
two_class_maxsub=two_class_pt.loc[two_class_pt.groupby('subject_id')['final_bin'].idxmax(),:]

time: 1.33 s


In [50]:
# 70/30 train/test set split with 12345=seed, splitting on max final bin of each SUBJECT_ID
train, test = train_test_split(two_class_maxsub, test_size=0.3, random_state=12345, 
                               stratify=two_class_maxsub['final_bin'])
# generate list of each SUBJECT_ID in each split
train_subject=list(train['subject_id'])
test_subject=list(test['subject_id'])

#filtering big_df on train subjects and test subjects to get my train/test splits.
big_df_train= big_df.loc[big_df.loc[:,'subject_id'].isin(train_subject),:].copy()
big_df_test= big_df.loc[big_df.loc[:,'subject_id'].isin(test_subject),:].copy()
del big_df

time: 713 ms


In [51]:
# #converting venttype to category
# big_df.loc[big_df.loc[:,'label']=='vent_recieved','value']= big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].astype('category')
# #big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].apply(astype('category'))
# big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'] = pd.Categorical(big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'])

time: 1.1 ms


In [52]:
def category_mapper(big_df):
    #converting categories to integers
    mapper={'Mech':2 , 'Oxygen': 1, 'None': 0}
    big_df.loc[big_df.loc[:,'label']=='vent_recieved','value']=big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].replace(mapper).copy()

    #gender_index=pd.factorize(big_df.loc[big_df.loc[:,'label']=='gender','value'])[1]
    mapper={'F':0 , 'M': 1}
    big_df.loc[big_df.loc[:,'label']=='gender','value']=big_df.loc[big_df.loc[:,'label']=='gender','value'].replace(mapper).copy()

    #leukocyte_index=pd.factorize(big_df.loc[big_df.loc[:,'label']=='leukocyte','value'])[1]
    mapper={'Neg/Not_tested':0 , 'pos': 1}
    big_df.loc[big_df.loc[:,'label']=='leukocyte','value']=big_df.loc[big_df.loc[:,'label']=='leukocyte','value'].replace(mapper).copy()
    #     pd.factorize(big_df.loc[big_df.loc[:,'label']=='leukocyte','value'])[0] 

    #nitrite_index=pd.factorize(big_df.loc[big_df.loc[:,'label']=='nitrite','value'])[1]
    mapper={'Neg/Not_tested':0 , 'pos': 1}
    big_df.loc[big_df.loc[:,'label']=='nitrite','value']=big_df.loc[big_df.loc[:,'label']=='nitrite','value'].replace(mapper).copy()
    #     pd.factorize(big_df.loc[big_df.loc[:,'label']=='nitrite','value'])[0] 
    
    
    # changing pao2/fio2 ratio to a category
    """
    A PaO2/FiO2 ratio less than or equal to 200 is necessary for the diagnosis of acute respiratory distress syndrome by the AECC criteria.[6] 
    The more recent Berlin criteria defines mild ARDS at a ratio of <300.

    A PaO2/FiO2 ratio less than or equal to 250 is one of the minor criteria for severe community acquired pneumonia (i.e., possible indication for inpatient treatment).

    A PaO2/FiO2 ratio less than or equal to 333 is one of the variables in the SMART-COP risk score for intensive respiratory or vasopressor support in community-acquired pneumonia.
    """

    
    pd.to_numeric(big_df[big_df['label']=="pao2fio2ratio"]['value'])
    labels= ['0-200', '201-333', "334-475","476+"]
    bins = pd.IntervalIndex.from_tuples([(0, 200), (200, 333), (333, 475),(475,3000)])

    big_df.loc[big_df['label']=="pao2fio2ratio",'value']=pd.cut(pd.to_numeric(big_df.loc[big_df['label']=="pao2fio2ratio",'value']), bins,right=False, labels=labels)
    big_df.loc[big_df['label']=="pao2fio2ratio",'value'].value_counts()

    
    return(big_df)


time: 72.5 ms


In [53]:
big_df_train=category_mapper(big_df_train)
big_df_test=category_mapper(big_df_test)

time: 1.79 s


#### qc

In [54]:
big_df_train.loc[big_df_train.loc[:,'label']=='vent_recieved','value'].value_counts()

1    1997
2    1801
0     272
Name: value, dtype: int64

time: 65.3 ms


In [55]:
big_df_train.loc[big_df_train.loc[:,'label']=='gender','value'].value_counts()

1    2269
0    1851
Name: value, dtype: int64

time: 69.2 ms


In [56]:
big_df_train.loc[big_df_train.loc[:,'label']=='bands','value'].value_counts()

absent    3581
<10        332
>10        207
Name: value, dtype: int64

time: 69.6 ms


In [57]:
big_df_train.loc[big_df_train.loc[:,'label']=='pco2','value'].value_counts()

absent    2818
<50        939
>50        363
Name: value, dtype: int64

time: 71.6 ms


In [58]:
big_df_train.loc[big_df_train.loc[:,'label']=='gender','value'].value_counts()

1    2269
0    1851
Name: value, dtype: int64

time: 70 ms


In [59]:
big_df_test.loc[big_df_test.loc[:,'label']=='pao2fio2ratio','value'].value_counts()

(475, 3000]    1284
(333, 475]      177
(0, 200]        151
(200, 333]      145
Name: value, dtype: int64

time: 34.9 ms


In [60]:
big_df_train.loc[big_df_train.loc[:,'label']=='pao2fio2ratio','value'].value_counts()

(475, 3000]    3092
(0, 200]        362
(333, 475]      354
(200, 333]      312
Name: value, dtype: int64

time: 70 ms


## calc median/iqr for standardization
take all non-categorical variables for HEALTHY PATIENTS and calculate the median and IQR for them. then will use this to make z scores via: x-median/IQR

### june 13: changed sparse to catgorical

In [61]:
continuous=['daily_sofa',
            'lactate',
            'mingcs',
            'diasbp',
            'heartrate',
            'meanartpress',
            'resprate',
            'sysbp',
            'temperature',
            'hemoglobin',
            'platelet',
            'wbc',
            'calcium',
            'glucose',
            'ph',
            'bicarbonate',
            'bun',
            'chloride',
            'creatinine',
            'inr',
            'potassium',
            'ptt',
            'sodium',
            'bilirubin',
            'spo2',
            'sum_elix']


onetime=['first_admit_age','height','weight']

vaso_active=['phenylephrine',
            'norepinephrine',
            'vasopressin',
            'dobutamine',
            'dopamine',
            'epinephrine'] 

ordinal=[
            'leukocyte',
            'nitrite',
            'vent_recieved',
            'o2_flow',
            'rrt',
            'pao2fio2ratio',
            'cancer_elix',
            "any_vasoactives",
            'bands', #added 6/13/19
            'pco2' #added 6/13/19
]

categorical=[
            "ethnicity",
            'gender'
]


time: 13.6 ms


In [62]:
def median_label_fxn(big_df):
    global final_pt_df2
    
    healthy_pt=list(final_pt_df2[final_pt_df2['final_bin']=="C_neg/A_partial"]['icustay_id'])

    #filter to only healthy patients filter
    ##splitting big_df, making a copy and restricting it to all values that will be usd in standardizing
    healthy_df=big_df[big_df['label'].isin(continuous+onetime)].copy()
    healthy_df['value']= pd.to_numeric(healthy_df['value']) #converting to numeric 
    healthy_df=healthy_df[healthy_df['icustay_id'].isin(healthy_pt)].copy() #only numerical values for cneg/ab partial pt
    
    #finding the last 24 hours of each healthy patient.
    healthy_pt_end=pd.DataFrame(healthy_df.groupby("icustay_id")['delta'].max())
    healthy_pt_end["start"]=healthy_pt_end['delta']- pd.to_timedelta("1 day 00:00:00")
    healthy_pt_end=healthy_pt_end.rename(columns={"delta":"end"}).reset_index()
    healthy_df= pd.merge(healthy_df,healthy_pt_end, left_on="icustay_id", right_on="icustay_id", how="left" ) #now have the last 24 hours annotated for each patient as start and end.
    
    #calculating medians and iqr for each label based on healthy patient's last 24 hours in icu
    median_label=pd.DataFrame((healthy_df.groupby("label")['value'].median())).reset_index()
    median_label=median_label.rename(columns={'value':"median"})
    iqr_label=pd.DataFrame((healthy_df.groupby("label")['value'].quantile(0.75)-healthy_df.groupby("label")['value'].quantile(0.25))).reset_index()
    iqr_label=iqr_label.rename(columns={'value':"iqr"})
    median_label=pd.merge(median_label,iqr_label)  #final median df
    return(median_label)

def combine_vasoactives(big_noCat, median_label):
    global vaso_active
    ##combining vasoactives
    vaso_active_df=big_noCat[big_noCat['label'].isin(vaso_active)].groupby('icustay_id')['value'].max().reset_index()
    vaso_active_df['uom']="y/n"
    vaso_active_df['label']="any_vasoactives"
    vaso_active_df['delta']=pd.to_timedelta("0days")
    vaso_active_df['source']="any_vasoactives"

    vaso_active_df=pd.merge(vaso_active_df, final_pt_df2[["icustay_id",'subject_id',"t_0"]], how="left", left_on="icustay_id", right_on="icustay_id")
    vaso_active_df=pd.merge(vaso_active_df, median_label, how="left").fillna(0)
    vaso_active_df['standardize']=vaso_active_df['value']
    vaso_active_df['raw_value']=vaso_active_df['value']
    vaso_active_df.head()

    # # #grabing the rest of the variables not suitable for range or mean/std
    big_noCat=pd.concat([big_noCat, vaso_active_df], sort=False)
    return(big_noCat)

def standardization_fxn(big_df):
    global continuous, onetime, vaso_active, ordinal, categorical
    
    median_label=median_label_fxn(big_df)
    
    ### dataformatting: convert all dtypes to a numeric type that pereserves nan. 
    #splitting categorical, ordinal and continuous
    big_categorical= big_df.loc[big_df.loc[:,'label'].isin(categorical),:].copy() 

    #continuous and ordinal variables
    big_noCat= big_df.loc[big_df.loc[:,'label'].isin(continuous),:].copy() 
    big_noCat['value']= big_noCat['value'].apply(pd.to_numeric, args=('coerce',)) #instead of convert to float, may preserve nan's better. 
    
    ### adding a standardized value (x-median)/iqr  where median is of the last 24 hours in time window for culture neg/ ab partial patients 
    big_noCat=pd.merge(big_noCat, median_label, how="left") 
    #loging values
    big_noCat['median']= np.log(big_noCat['median']+1.0)
    big_noCat['iqr']= np.log(big_noCat['iqr']+1.0)
    big_noCat['raw_value']=big_noCat['value']
    big_noCat['value']=np.log(big_noCat['value']+1.0)
    
    big_noCat['standardize']=((big_noCat['value']-big_noCat['median'])/big_noCat['iqr']).fillna(0) #standardize is log standardized

    #making an ordinal df to concat on
    ord_df=big_df.loc[big_df.loc[:,'label'].isin(ordinal+vaso_active),:].copy()
    ord_df['raw_value']=ord_df['value']
    ord_df['standardize']=ord_df['value']
    ord_df['median']=None
    ord_df['iqr']=None

    big_noCat=pd.concat([big_noCat,ord_df], sort=False)
    
    #making all ordinal values in standardize equal to unstandardized
    big_noCat.loc[big_noCat['label'].isin(ordinal+vaso_active),'standardize']=big_noCat.loc[big_noCat['label'].isin(ordinal+vaso_active),'value']  #do i need to add sparse?
    #ig_noCat.head()
    
    big_noCat['standardize']= big_noCat['standardize'].apply(pd.to_numeric, args=('coerce',)) #errors occuring downstream due to not having numeric, trying this 4/18/19
    
    ##last step, combining vasoactives into 1 feature
    big_noCat=combine_vasoactives(big_noCat, median_label)  

    return(big_noCat, big_categorical)

time: 358 ms


In [63]:
noCat_df_train, cat_df_train = standardization_fxn(big_df_train)
noCat_df_test, cat_df_test = standardization_fxn(big_df_test)

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:36: FutureWarning: Passing integers to fillna is deprecated, will raise a TypeError in a future version.  To retain the old behavior, pass pd.Timedelta(seconds=n) instead.


time: 1min 4s


In [64]:
#noCat_df_train[noCat_df_train['label']=='any_vasoactives'].head()#.unique()

time: 694 µs


# aggregation1:
### clincally guided min/max

In [65]:
hi_value= [#'bands',
'bilirubin',
'bun',
'chloride',
'creatinine',
'daily_sofa',
'glucose',
'heartrate',
'inr',
'lactate',
#'pco2',
'potassium',
'ptt',
'resprate',
'temperature',
'weight', 'rrt', 
'phenylephrine', 'norepinephrine', 'vasopressin', 'dobutamine', 'dopamine', 'epinephrine',  #added this and removed individual vasoactive 5/3/19
'first_admit_age','leukocyte','nitrite','vent_recieved','o2_flow', 
'any_vasoactives', #added this and removed individual vasoactive 5/3/19
'sum_elix', #added 6/4/19
'cancer_elix'  ]#added 6/4/19

low_value=['bicarbonate',
'diasbp',
'hemoglobin',
'meanartpress',
'mingcs',
'ph',
'platelet',
'spo2',
'sysbp']

both_value=['calcium',
'sodium',
'wbc']

#important_ordinal=["any_vasoactives"]
important_onetime=['first_admit_age','weight','pao2fio2ratio', 'pco2','bands'] #added pco2 and bands here, removed them above 06/14/19


time: 12.9 ms


In [66]:
noCat_df_train['label'].unique()#noCat_df_train['label']=='any_vasoactives'

array(['daily_sofa', 'diasbp', 'glucose', 'heartrate', 'meanartpress',
       'resprate', 'spo2', 'sum_elix', 'sysbp', 'temperature',
       'bicarbonate', 'bun', 'chloride', 'creatinine', 'hemoglobin',
       'inr', 'platelet', 'potassium', 'ptt', 'sodium', 'wbc', 'calcium',
       'lactate', 'ph', 'bilirubin', 'mingcs', 'bands', 'cancer_elix',
       'dopamine', 'epinephrine', 'norepinephrine', 'phenylephrine',
       'rrt', 'vasopressin', 'vent_recieved', 'dobutamine', 'o2_flow',
       'pao2fio2ratio', 'pco2', 'leukocyte', 'nitrite', 'any_vasoactive'],
      dtype=object)

time: 90.3 ms


### running the min/max aggregations.



In [67]:
def merge_cat_agg(num_df, cat_df, ):
    """
    merges the categorical and aggregated dataframes together. 
    """
    
    worst_df=pd.merge(num_df, cat_df, left_on='icustay_id', right_on='icustay_id',how='left')
    worst_df['ethnicity']=worst_df['ethnicity'].astype("category")
    worst_df['gender']=worst_df['gender'].astype("category")
    
    return(worst_df)

def clin_agg(big_noCat, big_cat,big_df, values="standardize"):
    """
    clincally guided aggregations.
    
    values= choose here if wanna use standardization or raw values.
    
    note: getting two minor errors, could use some cleaning up at later date.
    """
    global hi_value, low_value, both_value, important_onetime
    #max aggregation for selected variables
    big_max= big_noCat.loc[big_noCat.loc[:,'label'].isin(hi_value),:]
    table = pd.pivot_table(big_max, values=values, columns='label', index=['icustay_id'],aggfunc=max, dropna=False)
    
    #min aggregation for selected variables
    big_min= big_noCat.loc[big_noCat.loc[:,'label'].isin(low_value),:]
    table2 = pd.pivot_table(big_min, values=values, columns='label', index=['icustay_id'],aggfunc=min, dropna=False)
    
    #max&min aggregation for selected variables
    big_both= big_noCat.loc[big_noCat.loc[:,'label'].isin(both_value),:]
    table3 = pd.pivot_table(big_both, values=values, columns='label', index=['icustay_id'],aggfunc=[max,min], dropna=False)
    
    #first left join all different continuous aggregations together. 
    worst_df=pd.merge(table.reset_index(), table2.reset_index(), how='left')
    worst_df=pd.merge(worst_df, table3.reset_index(), left_on='icustay_id', right_on='icustay_id',how='left')
    
    ### formatting categorical to wide format to match the tables/worst_df
    big_cat= big_cat.pivot(
    index='icustay_id',
    values='value',
    columns='label').reset_index() #need to convert to wide format. should be one row per patient per time. 
    
    ## merging the categorical and aggregated dataframes together. 
    worst_df=merge_cat_agg(worst_df, big_cat) #using max/min aggregates 

    #adding important one_time values to final aggregated
    agg_remaining= big_df.loc[big_df.loc[:,'label'].isin(important_onetime),:]
    agg_table2 = pd.pivot_table(agg_remaining, values='value', columns='label', index=['icustay_id'],aggfunc=[max], dropna=False) 
    agg_table2.columns = agg_table2.columns.get_level_values(1)
    agg_table2=agg_table2.reset_index()
    agg_table2.head()#.rename(columns={})

    worst_df=pd.merge(worst_df, agg_table2, how='left')
    return(worst_df)


time: 123 ms


In [68]:
worst_df_train=clin_agg(noCat_df_train, cat_df_train,big_df_train, values="standardize")

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/reshape/merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/generic.py:3812: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


time: 41.8 s


In [69]:
worst_df_test=clin_agg(noCat_df_test, cat_df_test, big_df_test, values="standardize")

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/reshape/merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/generic.py:3812: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


time: 16.8 s


In [70]:
worst_df_train.head()

,icustay_id,any_vasoactive,bilirubin,bun,cancer_elix,chloride,creatinine,daily_sofa,dobutamine,dopamine,...,"(min, calcium)","(min, sodium)","(min, wbc)",ethnicity,gender,bands,first_admit_age,pao2fio2ratio,pco2,weight
0,200030.0,1.0,NaN,0.0881052,0,0.0207853,0,0.138647,1,0,...,0.082467,0.003972,0.114634,black,1,absent,54.19,"(333, 475]",<50,113.6
1,200033.0,1.0,NaN,-0.0822292,1,0.0042337,-0.234465,-0.430677,0,0,...,-0.424307,-0.024447,-0.207628,white/nonhispanic,1,absent,67.14,"(475, 3000]",absent,74.0
2,200036.0,0.0,NaN,0.119501,0,0.0167035,0,-0.430677,0,0,...,NaN,0.007917,-0.174736,white/nonhispanic,1,absent,74.93,"(475, 3000]",absent,79.0
3,200061.0,0.0,1.46649,-0.101564,0,0,-0.0740006,0,0,0,...,NaN,-0.008031,0.192664,unknown/other,1,absent,45.75,"(475, 3000]",absent,250.0
4,200063.0,0.0,NaN,0.148081,0,0.0042337,0.632268,0.347709,0,0,...,-0.555683,-0.012090,-0.109319,unknown/other,1,absent,37.07,"(475, 3000]",absent,NaN


time: 30.9 ms


In [71]:
worst_df_train['pao2fio2ratio'].describe()

count            4120
unique              4
top       (475, 3000]
freq             3092
Name: pao2fio2ratio, dtype: object

time: 12.3 ms


In [72]:
list(worst_df_train)

['icustay_id',
 'any_vasoactive',
 'bilirubin',
 'bun',
 'cancer_elix',
 'chloride',
 'creatinine',
 'daily_sofa',
 'dobutamine',
 'dopamine',
 'epinephrine',
 'glucose',
 'heartrate',
 'inr',
 'lactate',
 'leukocyte',
 'nitrite',
 'norepinephrine',
 'o2_flow',
 'phenylephrine',
 'potassium',
 'ptt',
 'resprate',
 'rrt',
 'sum_elix',
 'temperature',
 'vasopressin',
 'vent_recieved',
 'bicarbonate',
 'diasbp',
 'hemoglobin',
 'meanartpress',
 'mingcs',
 'ph',
 'platelet',
 'spo2',
 'sysbp',
 ('max', 'calcium'),
 ('max', 'sodium'),
 ('max', 'wbc'),
 ('min', 'calcium'),
 ('min', 'sodium'),
 ('min', 'wbc'),
 'ethnicity',
 'gender',
 'bands',
 'first_admit_age',
 'pao2fio2ratio',
 'pco2',
 'weight']

time: 3.07 ms


In [73]:
worst_df_train['vasopressin'].value_counts()

0.0    4045
1.0      75
Name: vasopressin, dtype: int64

time: 5.74 ms


In [74]:
worst_df_train['bands'].value_counts()

absent    3581
<10        332
>10        207
Name: bands, dtype: int64

time: 4.37 ms


In [75]:
worst_df_train['pco2'].value_counts()

absent    2818
<50        939
>50        363
Name: pco2, dtype: int64

time: 5.7 ms


In [76]:
date

'16052019'

time: 2.4 ms


In [77]:
##saving
pd.DataFrame(worst_df_train).to_csv(Path(
    wd+'/data/processed/merged/{}_worst_df_train_preImp_{}.csv'.format(date,timewindowdays)),index=False)
pd.DataFrame(worst_df_test).to_csv(Path(
    wd+'/data/processed/merged/{}_worst_df_test_preImp_{}.csv'.format(date,timewindowdays)),index=False)

time: 338 ms


In [79]:
wd+'/data/processed/merged/{}_worst_df_train_preImp_{}.csv'.format(date,timewindowdays)

'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/processed/merged/16052019_worst_df_train_preImp_48.csv'

time: 2.02 ms


In [ ]:
print(wd+'/data/processed/merged/{}_worstdf_preImp{}.csv'.format(date,timewindowdays))

# aggregation 2:
## out of date and needs updating as of 6/13/19
## 4-11-19: range, mean and stdev aggregations (newagg)

np.ptp = difference btween min and max


In [ ]:
##choose here if wanna use standardization or raw values.
values="standardize"

In [ ]:
big_range= big_noCat.loc[big_noCat.loc[:,'label'].isin(continuous),:]
table4 = pd.pivot_table(big_range, values=values, columns='label', index=['icustay_id'],aggfunc=[np.ptp, np.median, np.std], dropna=[False,False,False]) 
table4.columns=['_'.join(col).strip() for col in table4.columns.values]
table4=table4.reset_index()

In [ ]:
#grabing the rest of the variables not suitable for range or mean/std
remaining= big_noCat.loc[big_noCat.loc[:,'label'].isin(onetime+ordinal),:]
table5 = pd.pivot_table(remaining, values='value', columns='label', index=['icustay_id'],aggfunc=[max], dropna=False) 

In [ ]:
#table5.columns
table5.columns = table5.columns.get_level_values(1)
table5=table5.reset_index()

In [ ]:
table4.head()#.rename(columns={})

In [ ]:
## merging aggregations together
#first left join all different newagg together. 
table4=pd.merge(table4, table5, how='left')

In [ ]:
list(table4)[1:20]

In [ ]:
### formatting categorical to wide format to match the aggregated numerical
big_categorical= big_df.loc[big_df.loc[:,'label'].isin(categorical),:].copy()
big_categorical= big_categorical.pivot(
    index='icustay_id',
    values='value',
    columns='label').reset_index() #need to convert to wide format. should be one row per patient per time. 

### merging categorical with aggregated

In [ ]:
#using fxn written for worst_df to make newagg_df
newagg_df=merge_cat_agg(table4, big_categorical, twoclass=False) #using range/mean/stdev aggregates 


In [ ]:
newagg_df.head()

In [ ]:
list(newagg_df)

In [ ]:
pd.DataFrame(newagg_df).to_csv(Path(
    wd+'/data/processed/merged/%s_newaggdf_std.csv' %(date)),index=False)

In [ ]:
newagg_df['leukocyte']

In [ ]:
wd+'/data/processed/merged/%s_newaggdf_std.csv'

# aggregation 3:
# grabbing min/max/median/sd for only values found to be important from combining variable importance for random forest, boosted trees, and logistic regression on the baseline clinical max/min data.


In [ ]:
# important_variables=["temperature","sysbp","daily_sofa","meanartpress","bun","platelet","ptt","heartrate","pao2fio2ratio","creatinine","first_admit_age"]
important_continuous=["temperature","sysbp","daily_sofa","meanartpress","bun","platelet","ptt","heartrate","creatinine"]
important_ordinal=["any_vasoactives", 'vent_recieved', "leukocyte","pao2fio2ratio"]
important_onetime=['first_admit_age','weight']



In [ ]:
# #continuous and ordinal variables
# big_noCat= big_df.loc[big_df.loc[:,'label'].isin(continuous+ordinal),:].copy()
# big_noCat['value']= big_noCat['value'].apply(pd.to_numeric, args=('coerce',)) #instead of convert to float, may preserve nan's better. 

In [ ]:
big_noCat[big_noCat['label'].isin(vaso_active)].groupby('icustay_id')['value'].max().reset_index()

In [ ]:
##combining vasoactives
vaso_active_df=big_noCat[big_noCat['label'].isin(vaso_active)].groupby('icustay_id')['value'].max().reset_index()
vaso_active_df['uom']="y/n"
vaso_active_df['label']="any_vasoactives"
vaso_active_df['delta']=pd.to_timedelta("0days")
vaso_active_df['source']="any_vasoactives"

vaso_active_df=pd.merge(vaso_active_df, final_pt_df2[["icustay_id",'subject_id',"t_0"]], how="left", left_on="icustay_id", right_on="icustay_id")
vaso_active_df=pd.merge(vaso_active_df, median_label, how="left").fillna(0)
vaso_active_df['standardize']=vaso_active_df['value']
vaso_active_df['raw_value']=vaso_active_df['value']
vaso_active_df.head()

In [ ]:
vaso_active_df.iloc[3,1]

In [ ]:
big_noCat.head()

In [ ]:
big_noCat['label'].unique()
#big_noCat=pd.concat([big_noCat, vaso_active_df], sort=False)

In [ ]:
big_noCat[big_noCat['label']=="any_vasoactives"].head() #so this works

In [ ]:
##choose here if wanna use standardization or raw values.
values="standardize"
#big_noCat[values]= big_noCat[values].apply(pd.to_numeric, args=('coerce',)) #instead of convert to float, may preserve nan's better.

In [ ]:
# big_df2=pd.concat([big_df, vaso_active_df], sort=False)

# #splitting categorical, ordinal and continuous
# big_categorical= big_df.loc[big_df.loc[:,'label'].isin(important_ordinal+important_onetime),:].copy()

# #continuous and ordinal variables
# df_continuous= big_df.loc[big_df.loc[:,'label'].isin(important_continuous+ordinal),:].copy()
# big_noCat['value']= big_noCat['value'].apply(pd.to_numeric, args=('coerce',)) #instead of convert to float, may preserve nan's better. 

In [ ]:
df_continuous= big_noCat.loc[big_noCat.loc[:,'label'].isin(important_continuous),:]
agg_table1 = pd.pivot_table(df_continuous, values=values, columns='label', index=['icustay_id'],aggfunc=[np.max, np.min, np.median, np.std], dropna=[False,False,False, False]) 
agg_table1.columns=['_'.join(col).strip() for col in agg_table1.columns.values]
agg_table1=agg_table1.reset_index()

In [ ]:
#code above is not 

In [ ]:
#grabing the rest of the variables not suitable for range or mean/std
big_df2=pd.concat([big_df, vaso_active_df], sort=False)

agg_remaining= big_df2.loc[big_df2.loc[:,'label'].isin(important_onetime+important_ordinal),:]
agg_table2 = pd.pivot_table(agg_remaining, values='value', columns='label', index=['icustay_id'],aggfunc=[max], dropna=False) 
agg_table2.columns = agg_table2.columns.get_level_values(1)
agg_table2=agg_table2.reset_index()
agg_table2.head()#.rename(columns={})

agg_cont_cat=pd.merge(agg_table1, agg_table2, how='left')

In [ ]:
## merging aggregations together
#first left join all different newagg together. 
agg_cont_cat=pd.merge(agg_table1, agg_table2, how='left')

In [ ]:
list(agg_cont_cat)

In [ ]:
agg_cont_cat['pao2fio2ratio'].value_counts() 

In [ ]:
agg_cont_cat['any_vasoactives'].value_counts() 

In [ ]:
agg_cont_cat.head()

In [ ]:
pd.DataFrame(agg_cont_cat).to_csv(Path(
    wd+'/data/processed/merged/%s_newaggdf2_std.csv' %(date)),index=False)

In [ ]:
#YUS, got this reprocessed using a few of the top variables  4-16-19

# trying to pipeline this

In [ ]:
##sklearn example  https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html#sphx-glr-auto-examples-compose-plot-column-transformer-mixed-types-py
# We will train our classifier with the following features:
# Numeric Features:
# - age: float.
# - fare: float.
# Categorical Features:
# - embarked: categories encoded as strings {'C', 'S', 'Q'}.
# - sex: categories encoded as strings {'female', 'male'}.
# - pclass: ordinal integers {1, 2, 3}.

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_features = ['age', 'fare']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['embarked', 'sex', 'pclass']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression(solver='lbfgs'))])

X = data.drop('survived', axis=1)
y = data['survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

In [ ]:
# We create the preprocessing pipelines for both numeric and categorical data.
numeric_features = ['age', 'fare']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['embarked', 'sex', 'pclass']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [ ]:
class onehot(TransformerMixin):
    def __init__(self, cols_to_transform):
        self.cols_to_transform=cols_to_transform
        
    def transform(self,df ):
        data = pd.get_dummies(df, columns = self.cols_to_transform, drop_first=True)
        return(data)
    
    def fit(self, *_):
        return self